### Add Imports

In [13]:
import numpy as np
import copy
import os
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Read dataset and print the shapes

In [14]:
df = pd.read_csv('train.csv')

feature_names = df.columns
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values

In [15]:
print("Feature Names: ")
print(feature_names)
print("Features Matrix (X):")
print(X[:2])
print("Feature vector size: ")
print(X.shape)
print("\nTarget Vector (Y):")
print(Y[:2])

Feature Names: 
Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCo

### See features

##### See Unique values per column

In [16]:
def create_ranges(series, num_bins=20):
    min_val, max_val = series.min(), series.max()
    range_step = (max_val - min_val) / num_bins
    ranges = [f"{min_val + range_step * i:.2f} - {min_val + range_step * (i+1):.2f}" for i in range(num_bins)]
    return ranges

def print_ranges(df):
  for column in df.columns:
      unique_values = df[column].unique()

      if unique_values.size > 20 and np.issubdtype(df[column].dtype, np.number):
            print(f"Column '{column}' has more than 20 unique numeric values. Creating ranges:")
            print(create_ranges(df[column]))
      else:
        print(f"Unique values in '{column}':")
        print(unique_values)
      print()

#print_ranges(df)

##### Check for Nans

In [17]:
def replace_nans(df):
  df_numeric = df.select_dtypes(include=[np.number])
  df[df_numeric.columns] = df_numeric.fillna(0)
  return df

df = replace_nans(df)

### Feature Engineering

In [18]:
def preprocessDataset(df):
  numerical_data = df.select_dtypes(include=['int64', 'float64'])
  categorical_data = df.select_dtypes(include=['object'])

  sale_price_col = pd.DataFrame()
  if 'SalePrice' in df.columns:
    sale_price_col = df.SalePrice


  # One hot encoding for categories
  categorical_data_encoded = pd.get_dummies(categorical_data)

  # Apply Standard Scaling to numerical data
  scaler = StandardScaler()
  numerical_data_scaled = scaler.fit_transform(numerical_data)
  numerical_data_scaled_df = pd.DataFrame(numerical_data_scaled, columns=numerical_data.columns)

  preprocessed_df = pd.concat([numerical_data_scaled_df, categorical_data_encoded], axis=1)
  preprocessed_df.drop('Id', axis=1, inplace=True)
  if 'SalePrice' in df.columns:
    preprocessed_df.drop('SalePrice', axis=1, inplace=True)
  return preprocessed_df, sale_price_col

preprocessed_df, Y = preprocessDataset(df)


In [19]:
preprocessed_X = preprocessed_df.iloc[:, :].values


In [20]:
print(preprocessed_X.shape)

print("examples preprocessed_X:")
print(preprocessed_X[:2])

print(Y.shape)
print("examples of Y: ")
print(Y[:2])

(1460, 288)
examples preprocessed_X:
[[ 0.07337496  0.2128772  -0.20714171  0.65147924 -0.51719981  1.05099379
   0.87866809  0.51410389  0.57542484 -0.28865283 -0.94459061 -0.45930254
  -0.79343379  1.16185159 -0.12024172  0.37033344  1.10781015 -0.24106104
   0.78974052  1.22758538  0.16377912 -0.21145358  0.91220977 -0.95122649
   0.29602618  0.31172464  0.35100032 -0.75217584  0.21650316 -0.3593249
  -0.11633929 -0.27020835 -0.06869175 -0.08768781 -1.5991111   0.13877749
   0.          0.          0.          1.          0.          0.
   1.          0.          0.          0.          0.          0.
   1.          0.          0.          0.          1.          1.
   0.          0.          0.          0.          0.          1.
   1.          0.          0.          0.          0.          0.
   0.          0.          1.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.     

#### Train

In [21]:
input_shape = (preprocessed_X.shape[1],)
model = Sequential([
    Dense(8096, input_shape=input_shape),
    Dense(4048),
    Dense(2024),
    Dense(1024),
    Dense(512),
    Dense(256),
    Dense(1)
])

model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

In [ ]:
model.fit(preprocessed_X, Y, epochs=100, batch_size=32)


Epoch 1/100
46/46 [==============================] - 39s 823ms/step - loss: 6334774272.0000 - rmse: 79591.2969
Epoch 2/100
46/46 [==============================] - 38s 829ms/step - loss: 3370339072.0000 - rmse: 58054.6211
Epoch 3/100
46/46 [==============================] - 38s 823ms/step - loss: 1300145408.0000 - rmse: 36057.5273
Epoch 4/100
46/46 [==============================] - 37s 812ms/step - loss: 1117472512.0000 - rmse: 33428.6172
Epoch 5/100
46/46 [==============================] - 38s 817ms/step - loss: 1149692544.0000 - rmse: 33907.1172
Epoch 6/100
46/46 [==============================] - 37s 812ms/step - loss: 1290831232.0000 - rmse: 35928.1406
Epoch 7/100
46/46 [==============================] - 37s 797ms/step - loss: 1015829568.0000 - rmse: 31872.0820
Epoch 8/100
46/46 [==============================] - 37s 808ms/step - loss: 973159168.0000 - rmse: 31195.5000
Epoch 9/100
46/46 [==============================] - 37s 809ms/step - loss: 801589248.0000 - rmse: 28312.3516
Epo

# Predicton

In [ ]:
# Read train data

test_df = pd.read_csv('test.csv')

ids = test_df['Id'].values.astype(int)
test_df = replace_nans(test_df)
preprocessed_test_df, Y  = preprocessDataset(test_df)



In [ ]:
#print(preprocessed_test_df.shape)

unique_to_preprocessed_df = preprocessed_df.columns.difference(preprocessed_test_df.columns)
#print(unique_to_preprocessed_df)

# Populate difference of columns in test_Df
for column in unique_to_preprocessed_df:
    preprocessed_test_df[column] = 0

# maintain order of the columns too:
ordered_columns = [col for col in preprocessed_df.columns if col in preprocessed_test_df.columns]

preprocessed_test_df = preprocessed_test_df[ordered_columns]

# check difference again:
unique_to_preprocessed_df = preprocessed_df.columns.difference(preprocessed_test_df.columns)
#print(unique_to_preprocessed_df)

#print("Shape of train: " + str(preprocessed_df.shape) + " Shape of test: " + str(preprocessed_test_df.shape))



In [ ]:
predictions = model.predict(preprocessed_test_df)

46/46 [==============================] - 1s 14ms/step


In [ ]:
# concatenate predictions with ids and store to file.

result = np.column_stack((ids, predictions))

print("Predictions shape: " + str(predictions.shape))
print("ids shape: " + str(ids.shape))
print("result shape: " + str(result.shape))

Predictions shape: (1459, 1)
ids shape: (1459,)
result shape: (1459, 2)


In [ ]:

result_df = pd.DataFrame(result, columns=['Id', 'SalePrice'])
result_df['Id'] = result_df['Id'].astype(int)


# Save the DataFrame to a CSV file
result_df.to_csv('predictions.csv', index=False)
